In [ ]:
version


In [ ]:
# first intall the package - if installed # load
#install.packages("rgdal")
#install.packages("raster")
#install.packages("maptools")
#install.packages("ncdf4")
#install.packages("RColorBrewer")
#install.packages("maps")
#install.packages("lattice")
#install.packages("latticeExtra")
#install.packages("sf") 
#install.packages("rasterVis")
#install.packages("viridisLite")
#install.packages("viridis") 

In [1]:
# next load the packages
library(rgdal)
library(raster)
library(maptools)
library(ncdf4)
library(RColorBrewer)
library(maps)
library(lattice)
library(latticeExtra)
library(sf) 
library(rasterVis)
library(viridisLite)
library(viridis) 


Loading required package: sp

Please note that rgdal will be retired by the end of 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.

rgdal: version: 1.5-28, (SVN revision 1158)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Program Files/R/R-4.1.2/library/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Program Files/R/R-4.1.2/library/rgdal/proj
PROJ CDN enabled: FALSE
Linking to sp version:1.4-6
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading sp or rgdal.
Overwritten PROJ_LIB was C:/Program Files/R/R-4.1.2/library/rgdal/proj

Checking rgeos availability: TRUE
Please note that 'maptools' will be retired by the end of 2023,
plan transition at your

In [2]:
# this script is  for annual potential evaporation (PET) for catchments of Central Asia 
setwd("D:/R/CRU/")
getwd()

[1] "D:/R/CRU"

In [3]:
#Open netcdf files using brick function
p_1<-brick("D:/R/CRU/Data/cru_ts4.03.1981.1990.pre.dat.nc",varname="pre")
p_2<-brick("D:/R/CRU/Data/cru_ts4.03.1991.2000.pre.dat.nc",varname="pre")
p_3<-brick("D:/R/CRU/Data/cru_ts4.03.2001.2010.pre.dat.nc",varname="pre")

In [4]:
# each layer here is where PET is in mm a day for a month, so we multiply each layer iteratively by 30 days
# for all three layers (in this case) iteratively using loop
p_total<-30*p_1
n<-nlayers(p_total)
for(i in 1:nlayers(p_2))
{
  p_total[[n+i]]<-30*p_2[[i]]
}
n<-nlayers(p_total)
for(i in 1:nlayers(p_3))
{
  p_total[[n+i]]<-30*p_3[[i]]
}
nlayers(p_total)

[1] 360

In [5]:
# final big chunk of the code constructs a plot of mean annul total PET from 1981 to 2010 (30 years)
#NB: it shoudl be run in one go in Jupyter 
f_plot<-function(p,nm){
  #colors<-c(c('white'), rainbow(100))
  #c("brown2","white","cornflowerblue")
  #c('white','yellow','lightsteelblue3','lightsteelblue4','blue')
  #brewer.pal(n = 40, name = "RdYlBu")
  colors<-brewer.pal(n=15, name="RdYlBu")
  
  year<-as.numeric(substr(names(p),2,5))
  length(year)
  month<-as.numeric(substr(names(p),7,8))
  length(month)
  date<-ISOdate(year, month,1)
  p<-setZ(p, date)
  
  #for(i in 1:length(season))
  # season[i]<-paste(substr(season[i],2,5),f_season(as.numeric(substr(season[i],7,8))))
  #length(season)
  
  p<-setZ(p,year, name='year')
  p_year<-zApply(p, by=year,fun = sum)
  #y_year<-as.character(names(p_year)) 
  #for(i in 1:length(y_year))
  #  y_year[i]<-substr(season[i],6,11)
  # print(y_season)
  
  print(p_year)
  
  pp_year<-zApply(p_year, by=0,fun = mean)
  #plot(pp_year,
  #    xlim = c(40, 95),
  #   ylim = c(30, 60),
  #  col=colors
  #zlim=c(0,300),
  #col=colorRampPalette(c('lightsteelblue4','lightsteelblue3','lightsteelblue2','lightsteelblue', 'white', 'tomato', 'tomato2', 'tomato3', 'tomato4'))(1000)
  #)
  
  #for(s in 1:nlayers(pp_year))
  
  #{
  png("PET_Y_catchment.png", width = 13.8, height = 8, units = 'in', res = 500)
  par(
    cex=1.4,
    mar=c(4.7,4.1,4.7,7)
    #mgp=c(5,1,3)
  )
  plot(
    pp_year,
    main=nm,
    #ann=FALSE,
    #type='n',
    #xaxt='n',
    #yaxt='n',
    xlim = c(55, 100), 
    ylim = c(33, 50),
    zlim= c(0, 1800),
    #xlab="longitude",
    #ylab="latitude",
    col=colors,
    #scales=list(0,25,50,75,100,125,150,175,200,225,250),
    #zlim =c(0,300),
    #col = heat.colors(50)
    #scale=c(0,250)
    #col=colorRampPalette(rev(colors))(1000)
  )
  # ggsave("test.png", units="in", dpi=300, compression = 'lzw')
  
  #world_map<-readShapePoly("D:/R/CRU/SHP/TM_WORLD_BORDERS-0.2")
  #plot(world_map, add=T,
  #xlim = c(45, 88), 
  #ylim = c(35, 57)
  #)
  Alakol<-readShapePoly("D:/R/CRU/SHP/Alakol_basin_HS")
  plot(Alakol, add=T)
  Amu_darya<-readShapePoly("D:/R/CRU/SHP/amu_darya_basin")
  plot(Amu_darya, add=T)
  #Chu<-readShapePoly("D:/R/CRU/SHP/chu_river_basin_HS")
  #plot(Chu, add=T)
  Chu_talas<-readShapePoly("D:/R/CRU/SHP/chu_talas_river_basins_HS")
  plot(Chu_talas, add=T)
  #Helmand<-readShapePoly("D:/R/CRU/SHP/CRU_data/Helmand_HS")
  #plot(Helmand, add=T)
  Ile_b_a<-readShapePoly("D:/R/CRU/SHP/Ili-Balkhash_Alakol_HS")
  plot(Ile_b_a, add=T)
  Ile_b<-readShapePoly("D:/R/CRU/SHP/Ili-Balkhash_HS")
  plot(Ile_b, add=T)
  #Issyk_kul<-readShapePoly("D:/R/CRU/SHP/Issyk-Kul_basin_HS")
  #plot(Issyk_kul, add=T)
  syr_darya<-readShapePoly("D:/R/CRU/SHP/syr_darya_basin")
  plot(syr_darya, add=T)
  Tarim<-readShapePoly("D:/R/CRU/SHP/Tarim_HS")
  plot(Tarim, add=T)
  
  dev.off()
  
}




In [6]:
# here come the plot
f_plot(p_total,"")

Warning message in brewer.pal(n = 15, name = "RdYlBu"):
"n too large, allowed maximum for palette RdYlBu is 11
Returning the palette you asked for with that many colors
"


class      : RasterBrick 
dimensions : 360, 720, 259200, 30  (nrow, ncol, ncell, nlayers)
resolution : 0.5, 0.5  (x, y)
extent     : -180, 180, -90, 90  (xmin, xmax, ymin, ymax)
crs        : +proj=longlat +datum=WGS84 +no_defs 
source     : r_tmp_2022-03-19_120227_8284_12503.grd 
names      :  X1981,  X1982,  X1983,  X1984,  X1985,  X1986,  X1987,  X1988,  X1989,  X1990,  X1991,  X1992,  X1993,  X1994,  X1995, ... 
min values :      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0, ... 
max values : 255555, 244332, 209226, 276624, 270570, 283971, 215712, 266856, 245733, 264573, 232188, 235302, 222483, 218415, 222009, ... 
           : 1981, 2010 (min, max)



Warning message:
"readShapePoly is deprecated; use rgdal::readOGR or sf::st_read"
Warning message:
"readShapePoly is deprecated; use rgdal::readOGR or sf::st_read"
Warning message:
"readShapePoly is deprecated; use rgdal::readOGR or sf::st_read"
Warning message:
"readShapePoly is deprecated; use rgdal::readOGR or sf::st_read"
Warning message:
"readShapePoly is deprecated; use rgdal::readOGR or sf::st_read"
Warning message:
"readShapePoly is deprecated; use rgdal::readOGR or sf::st_read"
Warning message:
"readShapePoly is deprecated; use rgdal::readOGR or sf::st_read"


png 
  2